In [ ]:
!pip install wget

In [2]:
import pandas as pd
import numpy as np
import wget
import decimal
import os
from zipfile import ZipFile

#Acessando a base de dados da CVM

download da base itr (trimestral) e dfp (anual) desde 2011 até 2022

In [3]:
url_base_itr = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'
url_base_dfp = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'

Criando uma lista com todos os arquivos e baixando

In [4]:
arquivos_itr_zip = []
arquivos_dfp_zip = []
for ano in range(2011,2022):
  arquivos_itr_zip.append(f'itr_cia_aberta_{ano}.zip')
  arquivos_dfp_zip.append(f'dfp_cia_aberta_{ano}.zip')


In [5]:
for arq in arquivos_itr_zip:
  wget.download(url_base_itr+arq)

for arq in arquivos_dfp_zip:
  wget.download(url_base_dfp+arq)

In [6]:
for arq in arquivos_itr_zip:
  ZipFile(arq, 'r').extractall('ITR')

for arq in arquivos_dfp_zip:
  ZipFile(arq, 'r').extractall('DFP')

Concatenando todos os resultados em ITR e DFP e gerando CSV

In [7]:
# Criar a pasta
os.makedirs(os.path.join('/content', 'Dados'), exist_ok=True)

docs = ['BPA_ind','BPP_ind','DRE_ind','DVA_ind']

for doc in docs:
  arquivo = pd.DataFrame()
  for ano in range(2011,2022):
    arquivo = pd.concat([arquivo, pd.read_csv(f'ITR/itr_cia_aberta_{doc}_{ano}.csv', sep=';', decimal=',', encoding='ISO-8859-1')])
  arquivo.to_csv(f'Dados/itr_cia_aberta_{doc}_2011-2022',index=False)

for doc in docs:
  arquivo = pd.DataFrame()
  for ano in range(2011,2022):
    arquivo = pd.concat([arquivo, pd.read_csv(f'DFP/dfp_cia_aberta_{doc}_{ano}.csv', sep=';', decimal=',', encoding='ISO-8859-1')])
  arquivo.to_csv(f'Dados/dfp_cia_aberta_{doc}_2011-2022',index=False)

Carregando data frames

In [8]:
BPA_itr = pd.read_csv(f'/content/Dados/itr_cia_aberta_BPA_ind_2011-2022', sep=',', decimal=',')
BPP_itr = pd.read_csv(f'/content/Dados/itr_cia_aberta_BPP_ind_2011-2022', sep=',', decimal=',')
DVA_itr = pd.read_csv(f'/content/Dados/itr_cia_aberta_DVA_ind_2011-2022', sep=',', decimal=',')
DRE_itr = pd.read_csv(f'/content/Dados/itr_cia_aberta_DRE_ind_2011-2022', sep=',', decimal=',')
BPA_dfp = pd.read_csv(f'/content/Dados/dfp_cia_aberta_BPA_ind_2011-2022', sep=',', decimal=',')
BPP_dfp = pd.read_csv(f'/content/Dados/dfp_cia_aberta_BPP_ind_2011-2022', sep=',', decimal=',')
DVA_dfp = pd.read_csv(f'/content/Dados/dfp_cia_aberta_DVA_ind_2011-2022', sep=',', decimal=',')
DRE_dfp = pd.read_csv(f'/content/Dados/dfp_cia_aberta_DRE_ind_2011-2022', sep=',', decimal=',')

#Filtrando dados

filtrandos os dados trimestrais relevantes para BPA

In [9]:
BPA_itr = BPA_itr.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)
#BPA_itr = BPA_itr[BPA_itr['CD_CONTA'].str.len() <= 7]

#reodena colunas
BPA_itr = BPA_itr[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['Ativo Total','Ativo Circulante','Disponibilidades','Caixa e Equivalentes de Caixa']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPA_itr = BPA_itr.loc[BPA_itr['DS_CONTA'].isin(valores_a_manter)]

#separando os resultados
BPA_itr = BPA_itr[BPA_itr['ORDEM_EXERC'] == 'ÚLTIMO']
BPA_itr = BPA_itr.drop(['ORDEM_EXERC'],axis=1)

In [10]:
BPA_itr['CNPJ_CIA'].value_counts()

73.178.600/0001-18    124
01.616.929/0001-02    122
22.677.520/0001-76    121
07.040.108/0001-57    121
61.065.298/0001-02    120
                     ... 
41.096.674/0001-19      3
33.937.681/0001-78      3
67.915.785/0001-01      3
09.347.916/0001-97      3
24.410.913/0001-44      2
Name: CNPJ_CIA, Length: 1080, dtype: int64

In [11]:
BPA_dfp = BPA_dfp.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)
#BPA_itr = BPA_itr[BPA_itr['CD_CONTA'].str.len() <= 7]

#reodena colunas
BPA_dfp = BPA_dfp[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['Ativo Total','Ativo Circulante','Disponibilidades','Caixa e Equivalentes de Caixa']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPA_dfp = BPA_dfp.loc[BPA_dfp['DS_CONTA'].isin(valores_a_manter)]

#separando os resultados
BPA_dfp = BPA_dfp[BPA_dfp['ORDEM_EXERC'] == 'ÚLTIMO']
BPA_dfp = BPA_dfp.drop(['ORDEM_EXERC'],axis=1)

In [12]:
BPA_dfp['CNPJ_CIA'].value_counts()

61.065.298/0001-02    44
01.616.929/0001-02    44
73.178.600/0001-18    44
89.096.457/0001-55    42
09.116.278/0001-01    42
                      ..
10.846.981/0001-46     3
22.131.736/0001-31     3
24.410.913/0001-44     2
01.355.994/0001-21     2
16.838.421/0001-26     2
Name: CNPJ_CIA, Length: 1080, dtype: int64

filtrandos os dados trimestrais relevantes para BPP

In [13]:
BPP_itr = BPP_itr.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
BPP_itr = BPP_itr[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['Passivo Total','Passivo Circulante','Patrimônio Líquido']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPP_itr = BPP_itr.loc[BPP_itr['DS_CONTA'].isin(valores_a_manter)]

#separando os resultados
BPP_itr = BPP_itr[BPP_itr['ORDEM_EXERC'] == 'ÚLTIMO']
BPP_itr = BPP_itr.drop(['ORDEM_EXERC'],axis=1)

In [14]:
BPP_itr['CNPJ_CIA'].value_counts()

02.397.080/0001-96    102
15.101.405/0001-93    102
00.192.769/0001-59    102
02.291.077/0001-93    102
16.590.234/0001-76    102
                     ... 
41.096.674/0001-19      3
33.937.681/0001-78      3
67.915.785/0001-01      3
09.347.916/0001-97      3
24.410.913/0001-44      2
Name: CNPJ_CIA, Length: 1080, dtype: int64

In [15]:
BPP_dfp = BPP_dfp.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
BPP_dfp = BPP_dfp[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['Passivo Total','Passivo Circulante','Patrimônio Líquido']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPP_dfp = BPP_dfp.loc[BPP_dfp['DS_CONTA'].isin(valores_a_manter)]

#separando os resultados
BPP_dfp = BPP_dfp[BPP_dfp['ORDEM_EXERC'] == 'ÚLTIMO']
BPP_dfp = BPP_dfp.drop(['ORDEM_EXERC'],axis=1)

In [16]:
BPP_dfp['CNPJ_CIA'].value_counts()

61.082.004/0001-50    36
50.746.577/0001-15    36
08.764.621/0001-53    33
08.935.054/0001-50    33
51.466.860/0001-56    33
                      ..
60.631.090/0001-40     3
07.809.905/0001-56     3
87.870.952/0001-44     3
16.838.421/0001-26     2
24.410.913/0001-44     2
Name: CNPJ_CIA, Length: 1080, dtype: int64

filtrandos os dados trimestrais relevantes para DVA

In [17]:
DVA_itr = DVA_itr.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
DVA_itr = DVA_itr[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['Juros sobre o Capital Próprio','Dividendos']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
DVA_itr = DVA_itr.loc[DVA_itr['DS_CONTA'].isin(valores_a_manter)]

#separando os resultados
DVA_itr = DVA_itr[DVA_itr['ORDEM_EXERC'] == 'ÚLTIMO']
DVA_itr = DVA_itr.drop(['ORDEM_EXERC'],axis=1)

In [18]:
DVA_itr['CNPJ_CIA'].value_counts()

00.336.701/0001-04    77
45.987.245/0001-92    70
01.104.937/0001-70    70
00.359.742/0001-08    69
02.397.080/0001-96    68
                      ..
41.096.674/0001-19     2
33.937.681/0001-78     2
40.760.921/0001-77     2
51.468.056/0001-06     2
60.394.723/0001-44     2
Name: CNPJ_CIA, Length: 1080, dtype: int64

In [19]:
DVA_dfp = DVA_dfp.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
DVA_dfp = DVA_dfp[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['Juros sobre o Capital Próprio','Dividendos']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
DVA_dfp = DVA_dfp.loc[DVA_dfp['DS_CONTA'].isin(valores_a_manter)]

#separando os resultados
DVA_dfp = DVA_dfp[DVA_dfp['ORDEM_EXERC'] == 'ÚLTIMO']
DVA_dfp = DVA_dfp.drop(['ORDEM_EXERC'],axis=1)

In [20]:
DVA_dfp

,CNPJ_CIA,DENOM_CIA,CD_CVM,DT_REFER,DT_FIM_EXERC,ESCALA_MOEDA,CD_CONTA,DS_CONTA,VL_CONTA
87,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-12-31,2011-12-31,MIL,7.09.04.01,Juros sobre o Capital Próprio,3051308.0000000000
89,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-12-31,2011-12-31,MIL,7.09.04.02,Dividendos,1847625.0000000000
173,00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,2011-12-31,2011-12-31,MIL,7.09.04.01,Juros sobre o Capital Próprio,21612.0000000000
175,00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,2011-12-31,2011-12-31,MIL,7.09.04.02,Dividendos,319.0000000000
263,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,2011-12-31,2011-12-31,MIL,7.08.04.01,Juros sobre o Capital Próprio,0.0000000000
...,...,...,...,...,...,...,...,...,...
594647,96.418.264/0218-02,LOJAS QUERO QUERO S.A.,25038,2021-12-31,2021-12-31,MIL,7.08.04.02,Dividendos,0.0000000000
594723,97.191.902/0001-94,CONSERVAS ODERICH S.A.,4693,2021-12-31,2021-12-31,MIL,7.08.04.01,Juros sobre o Capital Próprio,12698.0000000000
594725,97.191.902/0001-94,CONSERVAS ODERICH S.A.,4693,2021-12-31,2021-12-31,MIL,7.08.04.02,Dividendos,0.0000000000
594803,97.837.181/0001-47,DEXCO S.A.,21091,2021-12-31,2021-12-31,MIL,7.08.04.01,Juros sobre o Capital Próprio,878401.0000000000


In [21]:
DVA_dfp['CNPJ_CIA'].value_counts()

01.104.937/0001-70    33
00.359.742/0001-08    27
71.476.527/0001-35    25
59.789.545/0001-71    25
50.746.577/0001-15    24
                      ..
33.147.315/0001-15     2
20.553.221/0001-02     2
03.392.498/0001-73     2
03.076.274/0001-52     2
87.870.952/0001-44     2
Name: CNPJ_CIA, Length: 1078, dtype: int64


filtrandos os dados trimestrais relevantes para DRE

In [22]:
DRE_itr = DRE_itr.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
DRE_itr = DRE_itr[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_INI_EXERC','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['3.01','3.03','3.05','3.07','3.11','3.13','3.99']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
DRE_itr = DRE_itr.loc[DRE_itr['CD_CONTA'].isin(valores_a_manter)]

#separando os resultados
DRE_itr = DRE_itr[DRE_itr['ORDEM_EXERC'] == 'ÚLTIMO']
DRE_itr = DRE_itr.drop(['ORDEM_EXERC'],axis=1)

In [23]:
DRE_itr['CNPJ_CIA'].value_counts()

30.306.294/0001-45    392
62.144.175/0001-20    385
65.654.303/0001-73    385
17.184.037/0001-10    385
33.040.601/0001-87    385
                     ... 
03.468.802/0001-19      6
33.066.408/0001-15      6
60.394.723/0001-44      6
26.643.756/0001-42      6
13.552.070/0001-02      6
Name: CNPJ_CIA, Length: 1080, dtype: int64

In [24]:
DRE_dfp = DRE_dfp.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
DRE_dfp = DRE_dfp[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_INI_EXERC','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['3.01','3.03','3.05','3.07','3.11','3.13','3.99']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
DRE_dfp = DRE_dfp.loc[DRE_dfp['CD_CONTA'].isin(valores_a_manter)]

#separando os resultados
DRE_dfp = DRE_dfp[DRE_dfp['ORDEM_EXERC'] == 'ÚLTIMO']
DRE_dfp = DRE_dfp.drop(['ORDEM_EXERC','DT_INI_EXERC'],axis=1)

In [25]:
DRE_dfp['CNPJ_CIA'].value_counts()

65.654.303/0001-73    77
91.669.747/0001-92    77
33.040.601/0001-87    77
02.682.287/0001-02    77
30.306.294/0001-45    77
                      ..
12.592.746/0001-10     6
01.287.588/0001-79     6
12.668.780/0001-21     6
12.668.792/0001-56     6
87.870.952/0001-44     6
Name: CNPJ_CIA, Length: 1079, dtype: int64

Padronizando a DRE ITR para resultados acumulados

In [26]:
# Converter as colunas datas para o tipo datetime
DRE_itr['DT_INI_EXERC'] = pd.to_datetime(DRE_itr['DT_INI_EXERC'])
DRE_itr['DT_FIM_EXERC'] = pd.to_datetime(DRE_itr['DT_FIM_EXERC'])
DRE_itr['DT_REFER'] = pd.to_datetime(DRE_itr['DT_REFER'])
# Calcular a diferença de tempo entre DT_INI_EXERC e DT_FIM_EXERC
DRE_itr['PERIODO'] = DRE_itr['DT_FIM_EXERC'] - DRE_itr['DT_INI_EXERC']
DRE_itr['PERIODO_DIF'] = DRE_itr['PERIODO'].diff()
# Substituindo 'NaT' por '0 days'
DRE_itr['PERIODO_DIF'] = DRE_itr['PERIODO_DIF'].fillna(pd.Timedelta(0))
# Eliminar as linhas de resultados trimestrais e manter os acumulados (como nos demais documentos)
DRE_itr = DRE_itr[DRE_itr['PERIODO_DIF'] >= '-2 days' ]
DRE_itr = DRE_itr.drop(['PERIODO_DIF','PERIODO'], axis=1)
DRE_itr = DRE_itr.drop('DT_INI_EXERC', axis = 1)

#Concatendando as bases de dados

In [27]:
df1 = pd.concat([BPA_itr,BPP_itr]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER'])
df2 = pd.concat([DRE_itr,DVA_itr]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER'])
df_itr = pd.concat([df1,df2]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER'])

df_itr['DT_FIM_EXERC']= pd.to_datetime(df_itr['DT_FIM_EXERC'])
df_itr['DT_REFER']= pd.to_datetime(df_itr['DT_REFER'])

df_itr = df_itr.sort_values(['CNPJ_CIA','DT_REFER'])

<ipython-input-27-e1c63037a91b>:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df2 = pd.concat([DRE_itr,DVA_itr]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER'])


In [28]:
df3 = pd.concat([BPA_dfp,BPP_dfp]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER'])
df4 = pd.concat([DRE_dfp,DVA_dfp]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER'])
df_dfp = pd.concat([df3,df4]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER'])

df_dfp['DT_FIM_EXERC']= pd.to_datetime(df_dfp['DT_FIM_EXERC'])
df_dfp['DT_REFER']= pd.to_datetime(df_dfp['DT_REFER'])

df_dfp = df_dfp.sort_values(['CNPJ_CIA','DT_REFER'])

In [29]:
df = pd.concat([df_itr,df_dfp]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER','CD_CONTA'])
df = df.reset_index(drop=True)
df = df.drop_duplicates()

#Filtrando a base de dados para empresas relevantes

In [30]:
df['CNPJ_CIA'].value_counts()

02.682.287/0001-02    660
59.285.411/0001-13    660
15.144.017/0001-90    660
65.654.303/0001-73    660
17.184.037/0001-10    660
                     ... 
09.347.916/0001-97     14
60.394.723/0001-44     14
03.820.456/0001-96     14
39.881.421/0001-04     14
16.838.421/0001-26     12
Name: CNPJ_CIA, Length: 1118, dtype: int64

In [31]:
df['DS_CONTA'].value_counts()

Patrimônio Líquido                                        28505
Passivo Total                                             28503
Dividendos                                                28482
Ativo Total                                               28471
Juros sobre o Capital Próprio                             28429
Passivo Circulante                                        28403
Ativo Circulante                                          28390
Lucro/Prejuízo do Período                                 28364
Caixa e Equivalentes de Caixa                             27318
Resultado Antes do Resultado Financeiro e dos Tributos    26712
Resultado Bruto                                           26710
Resultado Antes dos Tributos sobre o Lucro                26603
Lucro por Ação - (Reais / Ação)                           26600
Receita de Venda de Bens e/ou Serviços                    26524
Disponibilidades                                           1775
Lucro por Ação - (R$ / Ação)            

removendo empresas que não apresentam os resultados trimestralmente

In [32]:
contagem = df['CNPJ_CIA'].value_counts()
df = df[df['CNPJ_CIA'].isin(contagem[contagem > 42].index)]
# Iterar sobre os resultados da contagem
#for valor, count in contagem.items():
    #print(f"Valor: {valor}, Contagem: {count}")

mesclando nomes e removendo balanço irrelevantes

In [33]:
df['DS_CONTA'] = df['DS_CONTA'].replace({
    'Disponibilidades': 'Caixa',
    'Caixa e Equivalentes de Caixa': 'Caixa',
    'Lucro por Ação - (Reais / Ação)': 'Lucro por Ação',
    'Lucro por Ação - (R$ / Ação)': 'Lucro por Ação',
    'Lucro por Ação (R$/Ação)' : 'Lucro por Ação',
    'Lucro ou Prejuízo Líquido do Período' : 'Lucro/Prejuízo do Período',
    'a': 'Receitas',
    'Receita Bruta de Venda de Bens e/ou Serviços' : 'Receitas',
    'Receita de Venda de Bens e/ou Serviços' : 'Receitas',
    'Receita' : 'Receitas',
    'Receitas das Operações' : 'Receitas',
    'Receitas da Intermediação Financeira' : 'Receitas',
    'Receitas de Intermediação Financeira' : 'Receitas',
    'Resultado Antes Tributação/Participações' : 'Resultado Antes dos Tributos sobre o Lucro',
    'Resultado antes dos Tributos sobre o Lucro' : 'Resultado Antes dos Tributos sobre o Lucro',
    'Resultado Bruto Intermediação Financeira' : 'Resultado Bruto',
    'Resultado Bruto de Intermediação Financeira' : 'Resultado Bruto',
    'Resultado Operacional' : 'Resultado Antes do Resultado Financeiro e dos Tributos'

})

In [34]:
df = df[~df['DS_CONTA'].isin(['Reversão dos Juros sobre Capital Próprio','Outras Receitas e Despesas Operacionais','Resultado Líquido das Operações Continuadas', 'Lucro ou Prejuízo das Operações Continuadas'])]

Padronizandos os nomes dos balanços

In [35]:
df['DS_CONTA'] = df['DS_CONTA'].replace({
    'Resultado Antes do Resultado Financeiro e dos Tributos': 'EBITDA',
    'Resultado Antes dos Tributos sobre o Lucro': 'EBIT',
    'Juros sobre o Capital Próprio': 'JCP',
    'Lucro/Prejuízo do Período': 'Lucro Líquido',
    'Resultado Bruto': 'Lucro Bruto',
    'Lucro/Prejuízo do Período': 'Lucro',
    'Receitas': 'Receita Total'

})

<ipython-input-35-6dfe545ff298>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DS_CONTA'] = df['DS_CONTA'].replace({


#Transpor a coluna DS_CONTA para colunas individuais

eliminar colunas que não serão mais necessárias, mas para não perder informação da coluna "ESCALA_MOEDA" é necessário mulitplicar ou divididir a coluna "VL_CONTA"

In [36]:
df['VL_CONTA'] = df['VL_CONTA'].astype('float64')
pd.set_option('display.float_format', '{:.4f}'.format)
# Mapeamento para fatores multiplicativos
escala_mapping = {'UNIDADE': 1000}

# Função personalizada para multiplicar por fator de escala
def multiply_by_scale(row):
    escala = row['ESCALA_MOEDA']
    if escala in escala_mapping:
        return row['VL_CONTA'] / escala_mapping[escala]
    else:
        return row['VL_CONTA']

# Aplicar a função personalizada na coluna 'VL_CONTA'
df['VL_CONTA'] = df.apply(multiply_by_scale, axis=1)

df = df.drop(['DT_FIM_EXERC','CD_CONTA','ESCALA_MOEDA'], axis =1)

In [37]:
df = df.pivot_table(index=['CNPJ_CIA','DENOM_CIA','CD_CVM','DT_REFER'], columns='DS_CONTA', values='VL_CONTA' ).reset_index()
#reodena colunas
df = df[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER', 'Patrimônio Líquido', 'Ativo Total', 'Ativo Circulante', 'Passivo Total','Passivo Circulante', 'Receita Total',
         'Lucro Bruto','EBIT', 'EBITDA', 'Lucro', 'Caixa', 'Dividendos', 'JCP', 'Lucro por Ação']]
df = df.rename_axis(None, axis='columns')

#Inserindo Indicadores em um novo DF

In [38]:
# Vetor de indicadores
indicadores = ['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','VPA','LPA','PAYOUT','PL/ATIVOS',
                'PS/ATIVOS','DIV/PL','DIV/EBIT','DIV/EBITDA','M.EBITDA','ROE','ROIC','ROA','ILC','MLB','MLO','MLL','VLE']
# Criar novo DataFrame com coluna de indicadores repetidos
df2 = pd.DataFrame(columns=[indicadores])
df2 = df2.assign(**{coluna: df[coluna] for coluna in ['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER']})

Calculando os indicadores a partir dos balanços das empresas

In [48]:
df2['LPA'] = df['Lucro por Ação']
df2['VPA'] = df['Patrimônio Líquido'] / ( df['Lucro'] / df['Lucro por Ação'] )
df2['PAYOUT'] = ( df['Dividendos'] + df['JCP'] ) / ( df['Lucro'])

df2['PL/ATIVOS'] = df['Patrimônio Líquido'] / df['Ativo Total']
df2['PS/ATIVOS'] = df['Passivo Total'] / df['Ativo Total']
df2['DIV/PL'] = (df['Passivo Circulante'] - df['Caixa']) / df['Patrimônio Líquido']
df2['DIV/EBIT'] = (df['Passivo Circulante'] - df['Caixa']) / df['EBIT']
df2['DIV/EBITDA'] = (df['Passivo Circulante'] - df['Caixa']) / df['EBITDA']

df2['M.EBITDA'] = df['EBITDA'] / df['Receita Total']
df2['ROE'] = df['Lucro'] / df['Patrimônio Líquido']
df2['ROA'] = df['Lucro'] / df['Ativo Total']
df2['ROIC'] = df['EBIT'] / (df['Patrimônio Líquido'] + (df['Passivo Total'] - df['Passivo Circulante']))

df2['ILC'] = df['Ativo Circulante'] / df['Passivo Circulante']
df2['MLB'] = df['Lucro Bruto'] / df['Receita Total']
df2['MLO'] = df['EBIT'] / df['Receita Total']
df2['MLL'] = df['Lucro'] / df['Receita Total']

# Calcular a variação percentual do lucro
df2['VLE'] = df.groupby('CNPJ_CIA')['Lucro'].pct_change()
#df2['VLE'] = (df['Lucro'] / df['Lucro'].shift(1)) - 1

In [50]:
df2.head(10)

,CNPJ_CIA,DENOM_CIA,CD_CVM,DT_REFER,VPA,LPA,PAYOUT,PL/ATIVOS,PS/ATIVOS,DIV/PL,...,DIV/EBITDA,M.EBITDA,ROE,ROIC,ROA,ILC,MLB,MLO,MLL,VLE
0,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-03-31,0.0000,0.0000,0.4000,0.0662,1.0000,10.0505,...,118.6082,0.2191,0.0562,0.0146,0.0037,0.8585,0.3157,0.2197,0.1453,NaN
1,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-06-30,18.9728,2.1891,0.4000,0.0665,1.0000,9.8713,...,56.8123,0.2240,0.1154,0.0302,0.0077,0.8653,0.3207,0.2266,0.1488,1.1356
2,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-09-30,19.6745,3.2000,0.4000,0.0655,1.0000,9.7980,...,43.9160,0.1860,0.1626,0.0368,0.0107,0.8965,0.2678,0.1880,0.1356,0.4617
3,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-12-31,20.3217,4.2802,0.4000,0.0653,1.0000,9.6865,...,32.6106,0.1917,0.2106,0.0464,0.0138,0.9061,0.2732,0.1937,0.1360,0.3380
4,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-03-31,20.8758,0.8915,0.4000,0.0655,1.0000,9.4716,...,164.7938,0.1528,0.0427,0.0088,0.0028,0.9389,0.2823,0.1539,0.1135,-0.7914
5,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-06-30,21.6370,1.9390,0.4000,0.0644,1.0000,9.4589,...,77.7476,0.1595,0.0896,0.0180,0.0058,0.9202,0.2576,0.1609,0.1175,1.1750
6,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-09-30,22.2892,2.9188,0.4000,0.0632,1.0000,9.8968,...,58.3499,0.1561,0.1310,0.0256,0.0083,0.9017,0.2658,0.1580,0.1205,0.5048
7,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-12-31,22.9888,4.3023,0.4000,0.0627,1.0000,10.0228,...,42.2973,0.1718,0.1871,0.0378,0.0117,0.8738,0.2771,0.1854,0.1357,0.4724
8,00.000.000/0001-91,BCO BRASIL S.A.,1023,2013-03-31,21.6862,0.9057,0.4000,0.0559,1.0000,11.2306,...,199.4741,0.1650,0.0418,0.0076,0.0023,0.8781,0.3213,0.1654,0.1224,-0.7902
9,00.000.000/0001-91,BCO BRASIL S.A.,1023,2013-06-30,21.9443,3.5212,0.4000,0.0549,1.0000,11.6000,...,145.6190,0.1065,0.1605,0.0321,0.0088,0.8772,0.2486,0.3175,0.2145,2.8843


In [45]:
df.head(5)

,CNPJ_CIA,DENOM_CIA,CD_CVM,DT_REFER,Patrimônio Líquido,Ativo Total,Ativo Circulante,Passivo Total,Passivo Circulante,Receita Total,Lucro Bruto,EBIT,EBITDA,Lucro,Caixa,Dividendos,JCP,Lucro por Ação
0,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-03-31,52172709.0000,787908489.0000,460670797.0000,787908489.0000,536607202.0000,20179781.0000,6371709.0000,4432807.0000,4420967.0000,2932363.0000,12244142.0000,449024.0000,723921.0000,0.0000
1,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-06-30,54276101.0000,816131152.0000,479955357.0000,816131152.0000,554643276.0000,42097733.0000,13502837.0000,9539565.0000,9430620.0000,6262367.0000,18868357.0000,1044346.0000,1460601.0000,2.1891
2,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-09-30,56279126.0000,859280120.0000,512077755.0000,859280120.0000,571192875.0000,67494104.0000,18072581.0000,12687580.0000,12556373.0000,9153653.0000,19767405.0000,1405060.0000,2256401.0000,3.2000
3,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-12-31,58148690.0000,890352257.0000,518716710.0000,890352257.0000,572481618.0000,90080002.0000,24614257.0000,17448327.0000,17272140.0000,12247330.0000,9227217.0000,1847625.0000,3051308.0000,4.2802
4,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-03-31,59817963.0000,913866693.0000,545372613.0000,913866693.0000,580836511.0000,22500822.0000,6351239.0000,3463122.0000,3438060.0000,2554434.0000,14265629.0000,181408.0000,840365.0000,0.8915


In [52]:
df.to_csv('balanço_empresas_2011-2022.csv', index=True, header = True)
df2.to_csv('indicadores_empresas_2011-2022.csv', index=True, header = True)